## Deliverable 2. Create a Customer Travel Destinations Map.

In [31]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Catamarca,-28.47,-65.79,72.0,81,10,1.01,clear sky,AR,2020-10-23 03:56:17
1,1,Nouadhibou,20.93,-17.03,69.8,83,0,18.34,clear sky,MR,2020-10-23 03:56:17
2,2,Hobart,-42.88,147.33,64.4,48,75,5.82,broken clouds,AU,2020-10-23 03:53:21
3,3,Narsaq,60.92,-46.05,23.0,73,78,2.24,broken clouds,GL,2020-10-23 03:56:17
4,4,Vardo,70.37,31.11,35.6,95,75,26.40,broken clouds,NO,2020-10-23 03:56:17


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
1,1,Nouadhibou,20.93,-17.03,69.80,83,0,18.34,clear sky,MR,2020-10-23 03:56:17
2,2,Hobart,-42.88,147.33,64.40,48,75,5.82,broken clouds,AU,2020-10-23 03:53:21
5,5,Busselton,-33.65,115.33,66.00,53,0,3.00,clear sky,AU,2020-10-23 03:56:18
10,10,Albany,42.60,-73.97,57.00,85,86,1.23,overcast clouds,US,2020-10-23 03:56:18
15,15,Bealanana,-14.55,48.73,65.07,70,42,7.23,scattered clouds,MG,2020-10-23 03:56:20
17,17,Oxapampa,-10.58,-75.40,56.32,87,82,4.18,light rain,PE,2020-10-23 03:56:20
18,18,Camana,-16.62,-72.71,62.87,80,83,8.32,broken clouds,PE,2020-10-23 03:56:20
19,19,Castro,-24.79,-50.01,60.17,88,73,12.66,broken clouds,BR,2020-10-23 03:55:15
21,21,Cidreira,-30.18,-50.21,67.01,86,99,24.49,overcast clouds,BR,2020-10-23 03:56:21
22,22,Mataura,-46.19,168.86,61.00,70,100,3.00,light rain,NZ,2020-10-23 03:49:35


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                2
Date                   0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0)
clean_df.count()

City_ID                204
City                   204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
Country                204
Date                   204
dtype: int64

In [44]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nouadhibou,MR,69.80,clear sky,20.93,-17.03,
2,Hobart,AU,64.40,broken clouds,-42.88,147.33,
5,Busselton,AU,66.00,clear sky,-33.65,115.33,
10,Albany,US,57.00,overcast clouds,42.60,-73.97,
15,Bealanana,MG,65.07,scattered clouds,-14.55,48.73,
17,Oxapampa,PE,56.32,light rain,-10.58,-75.40,
18,Camana,PE,62.87,broken clouds,-16.62,-72.71,
19,Castro,BR,60.17,broken clouds,-24.79,-50.01,
21,Cidreira,BR,67.01,overcast clouds,-30.18,-50.21,
22,Mataura,NZ,61.00,light rain,-46.19,168.86,


In [45]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [48]:
# 7. Replace empty Hotel Name string with NaN
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.head()
#Drop rows in hotel dataframe with NaN values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Nouadhibou,MR,69.80,clear sky,20.93,-17.03,El Medina
2,Hobart,AU,64.40,broken clouds,-42.88,147.33,St Ives Hobart Accommodation Tasmania
5,Busselton,AU,66.00,clear sky,-33.65,115.33,Observatory Guest House
17,Oxapampa,PE,56.32,light rain,-10.58,-75.40,Hotel Heidinger
18,Camana,PE,62.87,broken clouds,-16.62,-72.71,Hostal Morales
...,...,...,...,...,...,...,...
664,Petropavlovsk-Kamchatskiy,RU,51.80,scattered clouds,53.05,158.65,"ООО ""Гостиничный комплекс ""Петропавловск"""
670,Pingdingshan,CN,64.54,clear sky,33.74,113.30,Chengfu Hotel
671,Nkowakowa,ZA,60.80,clear sky,-23.89,30.29,Moytel 966A Guest House & Spa
673,Zhob,PK,63.84,clear sky,31.34,69.45,ROOM # 105 DUBAI HOTEL ZHOB


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))